# Test

In [1]:
import requests
page = requests.get("https://stockbit.com/#/symbol/ICBP")
page

<Response [200]>

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
# scroll test
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")

In [8]:
from selenium.webdriver.common.by import By

driver.get("https://stockbit.com/symbol/ICBP")

comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text")

comments = list()

for index, element in enumerate(comment_elements):
    print(index, element.text, "\n")
    comments.append(element.text)

In [12]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

date = list()

for index, element in enumerate(date_elements):
    print(index, element.text, "\n")
    date.append(element.text)

0 15 Dec 22, 16:48 

1 15 Dec 22, 15:00 

2 15 Dec 22, 10:25 

3 14 Dec 22, 19:45 

4 14 Dec 22, 19:41 

5 14 Dec 22, 19:23 

6 14 Dec 22, 04:54 

7 13 Dec 22, 21:09 

8 13 Dec 22, 14:44 

9 13 Dec 22, 10:26 

10 13 Dec 22, 08:42 

11 12 Dec 22, 20:41 

12 12 Dec 22, 18:37 

13 12 Dec 22, 16:27 

14 12 Dec 22, 08:56 

15 12 Dec 22, 08:40 

16 11 Dec 22, 08:38 

17 11 Dec 22, 07:36 

18 11 Dec 22, 05:24 

19 10 Dec 22, 19:10 

20  



In [13]:
for a, b in zip([1,2], [3,4]):
    print(a,b)

1 3
2 4


In [10]:
from dateutil import parser
parser.parse("1 Oct 22, 15:00") >= parser.parse("1 Oct 22")

True

In [83]:
parser.parse("4 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29"[:16])

datetime.datetime(2022, 12, 4, 19, 42)

In [26]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

while True:
    # Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
    if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
    else:
        break
    

In [72]:
# No guarantee that Selenium will stop in 1 Oct 22.
date_elements[-2].text

'30 Sep 22, 09:22'

In [5]:
from random import randint

randint(5,10)

5

In [24]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

for symbol in symbols:
    print(f"https://stockbit.com/symbol/{symbol}")

https://stockbit.com/symbol/ICBP
https://stockbit.com/symbol/CPIN
https://stockbit.com/symbol/INDF
https://stockbit.com/symbol/MYOR
https://stockbit.com/symbol/CMRY
https://stockbit.com/symbol/GOOD
https://stockbit.com/symbol/MLBI


In [6]:
import pandas as pd

df = pd.DataFrame()
df['date'] = ['30 Sep 22, 09:22', '30 Sep 22, 09:23']
df['comment'] = ['$ICBP', '$ICBP2']
df['symbol'] = 'ICBP'

df_c = pd.DataFrame()

df_c['date'] = ['30 Sep 22, 09:22']
df_c['comment'] = ['$ICBP']
df_c['symbol'] = 'ICBP'

pd.concat([df, df_c])

,date,comment,symbol
0,"30 Sep 22, 09:22",$ICBP,ICBP
1,"30 Sep 22, 09:23",$ICBP2,ICBP
0,"30 Sep 22, 09:22",$ICBP,ICBP


In [46]:
for index, (a, b) in enumerate(zip([1,2], [3,4])):
    print(index, a, b)

0 1 3
1 2 4


# Scraping

In [29]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    # Handdle: IndexError: list index out of range
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

symbol ICBP date_elements 461 dates 460 comment_elements 461 comments 460 
symbol CPIN date_elements 201 dates 200 comment_elements 201 comments 200 
symbol INDF date_elements 401 dates 400 comment_elements 401 comments 400 
symbol MYOR date_elements 360 dates 359 comment_elements 380 comments 359 
symbol CMRY date_elements 120 dates 119 comment_elements 120 comments 119 
symbol GOOD date_elements 81 dates 80 comment_elements 81 comments 80 
symbol MLBI date_elements 101 dates 100 comment_elements 101 comments 100 


In [42]:
# <div class="stream-text-single">
#   <a class="ext-link-indicator postid-9756129" data-entity-id="9756129" href="http://bit.ly/3bLj4Oc">http://bit.ly/3bLj4Oc</a>
# </div class="stream-text-single">
# the <a class..> won't be included.
df.iloc[444]['comment']

'Lower Your Expectations\n\nKata - kata di atas saya kutip dari Charlie Munger.\n\nCharlie Munger itu kawan akrab Warren Buffett. Dia yang memperkenalkan konsep quality Investing ke Buffett. Sebelum akrab dengan Munger, Buffett adalah Grahamian sejati. Beli perusahaan di bawah tangible equities, jual ketika sudah capai fair price. Cari perusahaan yang sudah anjlok berat di bawah book value lalu jual ketika market apresiasi. Bahkan perusahaan yang tinggal 1 jumat pun bisa dibeli yang penting perusahaan itu masih punya nilai ketika dilikuidasi. Istilahnya cigarbutt atau investasi puntung rokok. Investor seperti pencari puntung rokok yang masih ada sisa dan masih nyala. Isap sekali dua kali, lalu buang.\n\nSejak kenal Munger dan baca bukunya Phil Fischer Common Stocks and Uncommon Profit, paradigma Buffett berubah.\n\nMunger dan Phil Fischer mengajarkan Buffett untuk investasi di perusahaan yang berkualitas dan punya potensi bertahan hingga 20-30 tahun ke depan. Dari pada capek - capek ja

In [44]:
df.to_csv("ICBP_MLBI.csv", index=False)

# Test

In [11]:
import pandas as pd
from dateutil import parser

df = pd.read_csv("./ICBP_MLBI.csv")

# df['date'] = df['date'].apply(lambda row: parser.parse(row[:16]))
df.head()

,date,comment,symbol
0,"15 Dec 22, 16:48",POLL\nSusu paling enak apa gan? Vox Populi Vox...,ICBP
1,"15 Dec 22, 15:00",$ICBP,ICBP
2,"15 Dec 22, 10:25",$GOTO main di 90 an aja ya kamu g usah kemana2...,ICBP
3,"14 Dec 22, 19:45",Info Singkat saham untuk 15 Desember 2022 (buk...,ICBP
4,"14 Dec 22, 19:41",@godbless @handusene sejak periode lockup open...,ICBP


In [51]:
def get_comment(index):
    print(df['symbol'][index])
    print(df['comment'][index])

In [44]:
def set_label(index: int, label_1: int, label_2: int):
    df['label 1'][index] = label_1
    df['label 2'][index] = label_2

In [204]:
# get index after ICBP
print(df['symbol'].ne('ICBP').idxmax())
print(df['symbol'][460])

460
CPIN


In [316]:
df['label 1'] = ''
df['label 2'] = ''
# index 0, sentiment neutral, type not spam
set_label(0, 0, 0)
# $ICBP
set_label(1, 0, 0)
# $GOTO main di 90 an aja ya kamu g usah kemana2, biar $ASII $TLKM $ADRO $ICBP dan bluechip yg lain bisa ngasih windress.
set_label(2, 0, 0)
# Info Singkat saham untuk 15 Desember 2022 (bukan rekomendasi jual-beli) :
set_label(3, 0, 0)
# @godbless @handusene sejak periode lockup open YU memang paling kenceng buang barang. Tapi hari ini YU gak manggung, CC jadi pemain utama.
set_label(4, 1, 0)
# $ULTJ fix tambah nyungsep liat chart kaya gini 🤯🤯
set_label(5, 1, 0)
# $HOKI yang merevolusi untuk menjadi $SRTG dikemudian hari, rasanya.
set_label(6, 2, 0)
# Risiko saham
set_label(7, 1, 0)
# Ko saya rada skeptis ya gaiss dengan kenaikan $GOTO hari ini. i mean, anjerr kalo beban yauda beban aja gt loo gausa ngesok naik segala terus besok2annya malah turun terus lagi..
set_label(8, 1, 0)
# $ICBP $INDF tes ombak ah...
set_label(9, 1, 0)
# Mirae Asset Sekuritas Indonesia Embun Pagi (13 Desember 2022)
set_label(10, 0, 0)
set_label(11, 2, 0)
set_label(12, 2, 0)
set_label(13, 2, 0)
set_label(14, 0, 0)
# IHSG Hari Ini Sulit Bangkit, Cek Saham BBRI, ICBP, hingga AKRA
set_label(15, 1, 0)
set_label(16, 2, 0)
set_label(17, 2, 0)
# Harga Kontainer Back To Normal
set_label(18, 0, 0)
# Join Zoom Meeting
set_label(19, 0, 1)
# Why KMDS ?
set_label(20, 2, 0)
set_label(21, 0, 0)
# @JS999 gw juga mau nuntut $ICBP mie soto gw g ada trauma makan mie soto,sekarang lebih milih makan mie goreng rendang
set_label(22, 0, 0)
# @JS999 makan Indomie lah ; biar $ICBP naik ..:)
set_label(23, 2, 0)
# Iya itulah, bukan hanya di Indonesia Pak. hampir disetiap negara kalau boleh transportasi itu gratis.
set_label(24, 0, 0)
set_label(25, 0, 0)
# Tertekan Akibat Harga Bahan Baku, Potensi Cuan Saham Indofood CBP (ICBP) Tetap Tinggi
set_label(26, 2, 0)
# Tertekan Akibat Harga Bahan Baku, Potensi Cuan Saham Indofood CBP (ICBP) Tetap Tinggi
set_label(27, 0, 0)
set_label(28, 0, 0)
set_label(29, 0, 0)
# $ICBP serius nanya parah suhu, udh lama ga lihat saham ini 3 bulanan bukan nya di 8 / 9rb.. sekarang koq naik ke 10rb ... ada apa gerangan ni , masih aman kah kalau masuk sekarang
set_label(30, 0, 0)
set_label(31, 0, 0)
set_label(32, 0, 0)
# ...
set_label(33, 0, 0)
set_label(34, 0, 0)
# $BBYB sesi 2 ARB kayanya
set_label(35, 1, 0)
# $ULTJ emang overhype IHSG turun kok malah naek sendiri
set_label(36, 1, 0)
# Stockpick : $ICBP Trading Buy TP 10.675
set_label(37, 2, 0)
# Kumpulan Artikel Pengaruh Jumlah Kepemilikan Investor Ritel terhadap Pergerakan Harga Saham
set_label(38, 0, 1)
# Pada tanggal 6 Desember 2022, $ICBP ditutup di titik tertingginya per hari ini di angka 10350, naik 0 poin (0,00%) dari harga closing kemarin. Secara teknikal dalam timeframe daily, harga ditutup di atas trend line di angka 10093,856. Sinyal BUY muncul sejak 7 November 2022, dan sinyal SPEC. SELL sempat muncul pada tanggal 18 November 2022 dan 22 November 2022.
set_label(39, 0, 0)
# Industry yg akan bertahan di akhir 2022 & Stockpick Q1 2023 yg mungkin udah banyak berseliweran apakah mungkin beli sekarang langsung naik ?
set_label(40, 1, 0)
# Mandiri Sekuritas: IHSG Sideways, Empat Saham Layak Dikoleksi Hari ini
set_label(41, 1, 0)
# izin belajar tentang $BOSS di @stockbit
#
#
#
#
# saya juga jualan produk indomie, kopi, gula dll. $ICBP $INDF $MYOR , boleh bapak/ibu berkenan beli dagangan saya. terima kasih
set_label(42, 0, 2)
set_label(43, 0, 0)
# @BimaAdiWiryo @RudiAlexander @Gungun08 @Tamami29 untuk saham $$INDF dan $ICBP prospeknya ud gak menarik karena selain labanya juga lagi turun, suka ada aksi2 yang merugikan pemegang saham seperti akuisisi pinehill yang bikin arus cashflow ke investor minoritas ngadat2 karena bayar hutang.. buat rekomendasi saham or buat yang mau nanya-nanya saham silahkan di follow ya
set_label(44, 1, 0)
# Skydrugz Corner: Emiten Dengan Goodwill Terbesar di Bursa
set_label(45, 1, 0)
set_label(46, 2, 0)
# @parsaulianpurba happy Cuan $ICBP
set_label(47, 2, 0)
# Analog switch off untuk jawa barat-tengah-timur sudah efektif awal desember ini.. another positive news buat emiten media kayak $SCMA ..
set_label(48, 2, 0)
set_label(49, 2, 0)
set_label(50, 0, 0)
set_label(51, 0, 0)
set_label(52, 0, 0)
set_label(53, 2, 0)
# kayaknya bentar lg ngoceh $INDF ma $ICBP ini abis makan indomie
set_label(54, 2, 0)
set_label(55, 1, 0)
# " Jangan membeli saham karna anda suka produknya, tapi beli lah saham karna anda paham bisnisnya maupun paham karakter dari ownernya "
set_label(56, 0, 0)
set_label(57, 2, 0)
set_label(58, 0, 0)
set_label(59, 0, 0)
# The essay for tech companies and tech adoption
set_label(60, 0, 1)
set_label(61, 0, 0)
set_label(62, 0, 0)
# Jenis Keunggulan Kompetitif
set_label(63, 0, 1)
set_label(64, 0, 0)
set_label(65, 0, 1)
set_label(66, 0, 1)
# Kendala modal dalam investasi
set_label(67, 0, 1)
set_label(68, 0, 0)
# @Elite saya tahun depan ingin fokus ke consumer goods, seperti $INDF, $ICBP $MYOR. karena mendekati tahun pemilu, biasanya suka ada bagi2 sembako/barang makanan lain. Diharapkan dapat meningkatkan sales.
set_label(69, 0, 0)
set_label(70, 0, 0)
set_label(71, 1, 0)
set_label(72, 2, 0)
set_label(73, 1, 0)
set_label(74, 0, 1)
set_label(75, 1, 0)
set_label(76, 1, 0)
set_label(77, 0, 0)
set_label(78, 1, 0)
set_label(79, 0, 0)
set_label(80, 0, 1)
set_label(81, 0, 0)
set_label(82, 1, 0)
set_label(83, 0, 0)
# $INDF $ICBP, lumayaaan...
# Revenue YOY Q3 naik 11%, QOQ naik 10,6%
# 
# Tapi net profitnya YOY turun (6,7 T vs 8 T) ?
# Iya karena kena beban keuangan sebesar 5,4 T dimana 3,1 T nya itu adalah RUGI NETO atas SELISIH NILAI TUKAR MATA UANG (lihat catatan kaki 31). Simplenya karena dollarnya sekarang ada di Rp. 15,727, kalau nanti dollar balik ke Rp 13,600 ya RUGI NETO 3,1 T ini semua hilang karena rugi kurs ini BELUM di REALIZED sampai jatuhnya tempo pembayaran obligasi INDF di 2031 dan 2051
# 
# Kalau rugi kurs ini kita abaikan, artinya net profit INDF harusnya 9,8 T... Dimana artinya naik sekitar 20% dibandingkan tahun lalu.. Yaa not bad lah...
set_label(84, 0, 0)
set_label(85, 0, 2)
set_label(86, 2, 0)
set_label(87, 0, 0)
set_label(88, 1, 0)
set_label(89, 2, 0)
set_label(90, 0, 0)
set_label(91, 0, 0)
set_label(92, 0, 2)
set_label(93, 0, 0)
set_label(94, 0, 2)
set_label(95, 2, 0)
set_label(96, 0, 2)
set_label(97, 0, 0)
set_label(98, 0, 0)
set_label(99, 0, 0)
# $ICBP selamat kepada top holding stockbit yang beli avg 10.000
set_label(100, 0, 0)
set_label(460, 2, 0)
set_label(461, 1, 0)
set_label(462, 0, 0)
set_label(463, 0, 0)
# NH Morning Brief – Dec. 14, 2022
set_label(464, 0, 1)
set_label(465, 0, 0)
# $JPFA $CPIN $CPRO $WMUU
# 
# Q4 biasanya revenue naik. Cocok di collect nih
set_label(466, 2, 0)
set_label(467, 0, 1)
set_label(468, 0, 0)
set_label(469, 2, 0)
set_label(470, 0, 2)
set_label(471, 0, 2)
set_label(472, 0, 2)
set_label(473, 1, 0)
set_label(474, 2, 0)
set_label(475, 1, 0)
set_label(476, 0, 0)
set_label(477, 0, 0)
# Nov_Des 2022 "seru" yah gocekan gerak harga saham. Nggak mau kalah dg gocekan Richarlison Brasil pada WorldCup kali ini... 😂
set_label(478, 0, 0)
set_label(479, 1, 0)
set_label(480, 0, 1)
set_label(481, 2, 0)
set_label(482, 1, 0)
# Raja Hutang semua itu kecuali $CPIN
set_label(483, 1, 0)
set_label(484, 0, 0)
# Skydrugz Corner: Emiten Pakan Ternak Undervalued?
# 
# Di postingan sebelumnya kita sudah bahas saham LQ45 https://cutt.ly/l1XlWpG
set_label(485, 0, 1)
set_label(486, 2, 0)
set_label(487, 2, 0)
# ayam-ayam 🐤
# 
# $JPFA kenapa lesu yah...
# 
# $MAIN $CPIN
# 
# memang semua sektor ada musimnya
set_label(488, 1, 0)
set_label(489, 0, 1)
set_label(490, 1, 0)
set_label(491, 1, 0)
set_label(492, 0, 0)
# $CPIN .. untuk ekspor bibit ayam sgt sulit, begitu ketat, byk uji laboratorium khusus nya mikrobiologi, dan saat ini cpin bs ekspor ke Singapura dan Timor Leste. emiten ini emang market leader nya di poultry, manajemen disini jd rujukan kompetitor nya, dan kebetulan saya pernah kerja disini. Oya liburan nataru bisa jd sentimen positif tuk poultry, selain inflasi yg akan melandai, harga komoditas yg mulai downtrend.
set_label(493, 2, 0)
set_label(494, 0, 0)
set_label(495, 0, 0)
set_label(496, 2, 0)
set_label(497, 1, 0)
# $CPIN break 5700 yuk
set_label(498, 2, 0)
set_label(499, 2, 0)
set_label(500, 0, 0)
set_label(501, 0, 0)
set_label(502, 0, 0)
# Have I ever told you
# I want you to the bone?
# Have I ever called you
# When you are all alone?
set_label(503, 0, 1)
set_label(504, 0, 0)
set_label(505, 0, 0)
# $WMUU melihat $CPIN dan $JPFA yg Q3 nya jatoh mending wns dlu tunggu konfirmasi Q3
set_label(506, 0, 0)
set_label(507, 0, 1)
set_label(508, 2, 0)
set_label(509, 0, 0)
# $WMUU jual emiten ini,

# ganti ke yang waras,

# $CPIN , $JPFA perusahaan bagus.
set_label(510, 1, 0)
set_label(511, 0, 0)
set_label(512, 2, 0)
set_label(513, 0, 0)
set_label(514, 0, 0)
set_label(515, 0, 0)
set_label(516, 0, 0)
set_label(517, 0, 0)
set_label(518, 0, 1)
set_label(519, 1, 0)
set_label(520, 0, 1)
set_label(521, 0, 1)
set_label(522, 0, 0)
set_label(523, 0, 0)
set_label(524, 0, 0)
set_label(525, 0, 0)
set_label(526, 1, 0)
set_label(527, 2, 0)
set_label(528, 0, 0)
set_label(529, 0, 2)
set_label(530, 0, 0)
set_label(531, 0, 0)
set_label(532, 1, 0)
set_label(533, 1, 0)
set_label(534, 0, 0)
set_label(535, 0, 0)
set_label(536, 0, 0)
set_label(537, 0, 0)
set_label(538, 0, 0)
set_label(539, 2, 0)
set_label(540, 0, 0)

In [317]:
df.to_csv("ICBP_MLBI_labeled.csv", index=False)